# Imports and Data Preparation

In [1]:
import os
import time
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, SubsetRandomSampler
from torch.quantization import (
    prepare_qat,
    convert,
    enable_observer,
    disable_observer,
    enable_fake_quant,
    disable_fake_quant,
)
from torchvision.models.quantization import resnet50
import numpy as np
from tqdm import tqdm

In [2]:
def get_cifar100_data_loaders(data_dir='./data'):
    train_tf = transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.5071, 0.4867, 0.4408), (0.2675, 0.2565, 0.2761)),
    ])
    test_tf = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5071, 0.4867, 0.4408), (0.2675, 0.2565, 0.2761)),
    ])

    train_ds = torchvision.datasets.CIFAR100(data_dir, train=True, download=True, transform=train_tf)
    test_ds = torchvision.datasets.CIFAR100(data_dir, train=False, download=True, transform=test_tf)

    train_size = len(train_ds)
    indices = list(range(train_size))
    np.random.seed(42)
    np.random.shuffle(indices)
    split = int(0.1 * train_size)
    train_idx, val_idx = indices[split:], indices[:split]

    train_loader = DataLoader(train_ds, batch_size=128, sampler=SubsetRandomSampler(train_idx), num_workers=4, pin_memory=True)
    val_loader = DataLoader(train_ds, batch_size=256, sampler=SubsetRandomSampler(val_idx), num_workers=4, pin_memory=True)
    test_loader = DataLoader(test_ds, batch_size=8, shuffle=False, num_workers=4, pin_memory=False)

    return train_loader, val_loader, test_loader, train_idx

def cutmix(data, targets, alpha=1.0):
    batch_size = data.size(0)
    indices = torch.randperm(batch_size)
    shuffled_data = data[indices]
    shuffled_targets = targets[indices]

    lam = np.random.beta(alpha, alpha)
    bbx1, bby1, bbx2, bby2 = rand_bbox(data.size(), lam)
    data[:, :, bbx1:bbx2, bby1:bby2] = shuffled_data[:, :, bbx1:bbx2, bby1:bby2]
    lam = 1 - ((bbx2 - bbx1) * (bby2 - bby1) / (data.size(-1) * data.size(-2)))

    return data, targets, shuffled_targets, lam

def rand_bbox(size, lam):
    W = size[2]
    H = size[3]
    cut_rat = np.sqrt(1. - lam)
    cut_w = int(W * cut_rat)
    cut_h = int(H * cut_rat)

    cx = np.random.randint(W)
    cy = np.random.randint(H)

    bbx1 = np.clip(cx - cut_w // 2, 0, W)
    bby1 = np.clip(cy - cut_h // 2, 0, H)
    bbx2 = np.clip(cx + cut_w // 2, 0, W)
    bby2 = np.clip(cy + cut_h // 2, 0, H)

    return bbx1, bby1, bbx2, bby2

train_loader, val_loader, test_loader, train_idx = get_cifar100_data_loaders()

Files already downloaded and verified
Files already downloaded and verified


# fp32 with quantized resnet model

In [6]:

def evaluate_accuracy_and_latency(model, loader, device, num_batches=None, desc="Evaluating"):
    model.eval()
    model.to(device)
    correct = total = 0
    latencies = []
    with torch.no_grad():
        for i, (imgs, labels) in enumerate(tqdm(loader, desc=desc, leave=False)):
            if num_batches is not None and i >= num_batches:
                break
            imgs, labels = imgs.to(device), labels.to(device)
            start_time = time.time()
            outputs = model(imgs)
            latency = (time.time() - start_time) * 1000  # Latency in ms/batch
            latencies.append(latency)
            preds = outputs.argmax(dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    accuracy = 100.0 * correct / total
    avg_latency = sum(latencies) / len(latencies)
    return accuracy, avg_latency

# Main training loop (same structure as provided, with CutMix logic from QAT code)
def main():
    train_device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    assert train_device.type == 'cuda', "GPU not available for training/validation"
    test_device = torch.device('cpu')

    # train_loader, val_loader, test_loader, train_idx = get_cifar100_data_loaders()

    model = resnet50(pretrained=False, quantize=False, num_classes=100)
    # Modify conv1 and maxpool for 32x32 inputs
    model.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=0, bias=False)
    model.maxpool = nn.Identity()  # Remove maxpool to preserve spatial dimensions
    model.to(train_device)

    optimizer = optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-4)
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=100)
    criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
    best_val_acc = 0.0
    best_model_path = 'best_fp32_resnet50_cifar100_32x32_100_cutmix.pth'
    epochs = 100
    patience = 20
    patience_counter = 0
    cutmix_prob = 0.5

    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        for imgs, targets in tqdm(train_loader, desc=f"Training Epoch {epoch+1}/{epochs}", leave=True):
            imgs, targets = imgs.to(train_device), targets.to(train_device)
            optimizer.zero_grad()
            if np.random.random() < cutmix_prob:
                imgs, targets_a, targets_b, lam = cutmix(imgs, targets, alpha=1.0)
                outputs = model(imgs)
                loss = lam * criterion(outputs, targets_a) + (1 - lam) * criterion(outputs, targets_b)
            else:
                outputs = model(imgs)
                loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * imgs.size(0)

        val_acc, _ = evaluate_accuracy_and_latency(model, val_loader, train_device, desc="Validating")
        print(f"Epoch {epoch+1}/{epochs} – Loss: {running_loss/len(train_idx):.4f}, Val Accuracy: {val_acc:.2f}%")

        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), best_model_path)
            print(f"Saved best FP32 model with Val Accuracy: {best_val_acc:.2f}%")
            patience_counter = 0
        else:
            patience_counter += 1
            print(f"No improvement in validation accuracy. Patience counter: {patience_counter}/{patience}")

        if patience_counter >= patience:
            print(f"Early stopping triggered after {epoch+1} epochs.")
            break

        scheduler.step()

    model.load_state_dict(torch.load(best_model_path))
    test_acc, avg_latency = evaluate_accuracy_and_latency(model, test_loader, test_device, desc="Testing FP32")
    print(f"\nBest FP32 Model – Test Accuracy: {test_acc:.2f}%")
    print(f"Average FP32 Inference Latency (CPU): {avg_latency:.2f} ms/batch")

if __name__ == '__main__':
    main()

Training Epoch 1/100: 100%|██████████| 352/352 [00:19<00:00, 17.63it/s]


Epoch 1/100 – Loss: 4.3153, Val Accuracy: 13.44%
Saved best FP32 model with Val Accuracy: 13.44%


Training Epoch 2/100: 100%|██████████| 352/352 [00:20<00:00, 17.53it/s]


Epoch 2/100 – Loss: 3.9347, Val Accuracy: 18.54%
Saved best FP32 model with Val Accuracy: 18.54%


Training Epoch 3/100: 100%|██████████| 352/352 [00:19<00:00, 17.67it/s]


...

Training Epoch 99/100: 100%|██████████| 352/352 [00:19<00:00, 17.66it/s]


Epoch 99/100 – Loss: 1.3812, Val Accuracy: 76.02%
No improvement in validation accuracy. Patience counter: 2/20


Training Epoch 100/100: 100%|██████████| 352/352 [00:20<00:00, 17.58it/s]


Epoch 100/100 – Loss: 1.3134, Val Accuracy: 76.42%
No improvement in validation accuracy. Patience counter: 3/20



Best FP32 Model – Test Accuracy: 76.69%
Average FP32 Inference Latency (CPU): 267.59 ms/batch


# ENT KD

In [13]:
import os
import time
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, SubsetRandomSampler
from torch.quantization import (
    prepare_qat,
    convert,
    enable_observer,
    disable_observer,
    enable_fake_quant,
    disable_fake_quant,
)
from torchvision.models.quantization import resnet50
import numpy as np
from tqdm import tqdm

torch.backends.quantized.engine = 'fbgemm'



class EntKDLoss(nn.Module):
    def __init__(self, base_temperature=3.0, alpha=0.7, beta=0.1):
        super(EntKDLoss, self).__init__()
        self.base_temperature = base_temperature
        self.alpha = alpha
        self.beta = beta
        self.kld_loss = nn.KLDivLoss(reduction='batchmean')

    def forward(self, student_outputs, teacher_outputs, targets_a, targets_b, lam):
        # Compute teacher entropy
        teacher_probs = F.softmax(teacher_outputs, dim=1)
        entropy = -torch.sum(teacher_probs * torch.log(teacher_probs + 1e-10), dim=1).mean()
        
        # Dynamic temperature
        temperature = self.base_temperature / (1 + self.beta * entropy)
        temperature = torch.clamp(temperature, min=1.0, max=10.0)
        
        # Compute mixed CE loss
        log_prob = F.log_softmax(student_outputs, dim=1)
        ce_loss = - (lam * log_prob[torch.arange(student_outputs.size(0)), targets_a] + 
                     (1 - lam) * log_prob[torch.arange(student_outputs.size(0)), targets_b]).mean()
        
        # Compute KLD loss
        kld = self.kld_loss(
            F.log_softmax(student_outputs / temperature, dim=1),
            F.softmax(teacher_outputs / temperature, dim=1)
        ) * (temperature ** 2)
        
        return self.alpha * ce_loss + (1 - self.alpha) * kld

def main():
    train_device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    assert train_device.type == 'cuda', "GPU not available for training/validation"
    test_device = torch.device('cpu')

    train_loader, val_loader, test_loader, train_idx = get_cifar100_data_loaders()

    # Load pre-trained FP32 teacher model
    teacher_model = resnet50(pretrained=False, num_classes=100)
    teacher_model.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=0, bias=False)
    teacher_model.maxpool = nn.Identity()
    teacher_model.load_state_dict(torch.load('best_fp32_resnet50_cifar100_32x32_100_cutmix.pth'))
    teacher_model.to(train_device)
    teacher_model.eval()

    # Initialize QAT student model
    student_model = resnet50(pretrained=False, quantize=False, num_classes=100)
    student_model.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=0, bias=False)
    student_model.maxpool = nn.Identity()
    student_model.fuse_model()
    student_model.qconfig = torch.quantization.get_default_qat_qconfig('fbgemm')
    prepare_qat(student_model, inplace=True)
    student_model.to(train_device)

    # Calibration
    student_model.eval()
    disable_fake_quant(student_model)
    enable_observer(student_model)
    with torch.no_grad():
        for i, (imgs, _) in enumerate(tqdm(train_loader, desc="Calibrating", leave=False)):
            if i >= 200:
                break
            imgs = imgs.to(train_device)
            student_model(imgs)
    disable_observer(student_model)
    enable_fake_quant(student_model)

    optimizer = optim.AdamW(student_model.parameters(), lr=1e-3, weight_decay=1e-4)
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=100)
    criterion = EntKDLoss(base_temperature=3.0, alpha=0.3, beta=0.1)
    best_val_acc = 0.0
    best_model_path = 'qat_entkd_cutmix_resnet50_cifar100_float_32x32_100.pth'
    epochs = 100
    patience = 20
    patience_counter = 0

    for epoch in range(epochs):
        student_model.train()
        enable_fake_quant(student_model)
        running_loss = 0.0
        for imgs, targets in tqdm(train_loader, desc=f"Training Epoch {epoch+1}/{epochs}", leave=False):
            imgs, targets = imgs.to(train_device), targets.to(train_device)
            if np.random.rand() < 0.5:  # Apply CutMix with 50% probability
                mixed_imgs, targets_a, targets_b, lam = cutmix(imgs, targets)
            else:
                mixed_imgs = imgs
                targets_a = targets
                targets_b = targets
                lam = 1.0
            optimizer.zero_grad()
            with torch.no_grad():
                teacher_outputs = teacher_model(mixed_imgs)
            student_outputs = student_model(mixed_imgs)
            loss = criterion(student_outputs, teacher_outputs, targets_a, targets_b, lam)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * imgs.size(0)

        val_acc, _ = evaluate_accuracy_and_latency(student_model, val_loader, train_device, desc="Validating")
        print(f"Epoch {epoch+1}/{epochs} – Loss: {running_loss/len(train_idx):.4f}, ValAcc: {val_acc:.2f}%")

        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(student_model.state_dict(), best_model_path)
            print(f"Saved best QAT+EntKD+CutMix model with ValAcc: {best_val_acc:.2f}%")
            patience_counter = 0
        else:
            patience_counter += 1
            print(f"No improvement in validation accuracy. Patience counter: {patience_counter}/{patience}")

        if patience_counter >= patience:
            print(f"Early stopping triggered after {epoch+1} epochs.")
            break

        scheduler.step()

    student_model.load_state_dict(torch.load(best_model_path))
    fake_quant_acc, fake_quant_latency = evaluate_accuracy_and_latency(student_model, test_loader, train_device, desc="Testing Fake-Quantized")
    print(f"\nBest Fake-Quantized Model – Test Accuracy: {fake_quant_acc:.2f}%")
    print(f"Average Fake-Quantized Inference Latency (GPU): {fake_quant_latency:.2f} ms/batch")

    student_model.cpu()
    student_model.eval()
    quantized_model = convert(student_model, inplace=False)

    int8_acc, int8_latency = evaluate_accuracy_and_latency(quantized_model, test_loader, test_device, desc="Testing INT8")
    print(f"Best INT8 Model – Test Accuracy: {int8_acc:.2f}%")
    print(f"Average INT8 Inference Latency (CPU): {int8_latency:.2f} ms/batch")

    torch.save(quantized_model.state_dict(), 'qat_entkd_cutmix_resnet50_cifar100_int8_final_32x32_100.pth')

if __name__ == '__main__':
    main()

Files already downloaded and verified
Files already downloaded and verified


Epoch 1/100 – Loss: 2.1634, ValAcc: 11.82%
Saved best QAT+EntKD+CutMix model with ValAcc: 11.82%


Epoch 2/100 – Loss: 1.9454, ValAcc: 19.42%
Saved best QAT+EntKD+CutMix model with ValAcc: 19.42%


Epoch 3/100 – Loss: 1.8255, ValAcc: 29.62%
Saved best QAT+EntKD+CutMix model with ValAcc: 29.62%


Epoch 4/100 – Loss: 1.6762, ValAcc: 34.98%
Saved best QAT+EntKD+CutMix model with ValAcc: 34.98%


...

Epoch 98/100 – Loss: 0.3102, ValAcc: 77.72%
No improvement in validation accuracy. Patience counter: 5/20


Epoch 99/100 – Loss: 0.3395, ValAcc: 77.78%
No improvement in validation accuracy. Patience counter: 6/20


Epoch 100/100 – Loss: 0.3269, ValAcc: 78.00%
No improvement in validation accuracy. Patience counter: 7/20



Best Fake-Quantized Model – Test Accuracy: 78.42%
Average Fake-Quantized Inference Latency (GPU): 25.87 ms/batch


Best INT8 Model – Test Accuracy: 78.40%
Average INT8 Inference Latency (CPU): 130.78 ms/batch
